# Trabajando con datos en pandas

Ya habiendo importado nuestros datos a pandas, nuestras siguientes tareas serán: limpieza de la base de datos, transformación de variables y cálculo de nuevos indicadores, consultas específicas (por agrupación, selección, filtrado, y reordenamiento), generación de reportes (en la forma de tablas o gráficos).  Antes que todo, podemos obtener un resumen de nuestra información con los métodos `.describe()` y `.info()`.

```python
import pandas as pd
sueño = pd.read_csv('https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/msleep.csv',
                      index_col = 0)
sueño.describe()
```


## 1. Fundamentos de limpieza de datos en pandas

### Datos faltantes (NA)

Lo primero que necesitamos en este rubro es reconocer los datos faltantes.  Pandas reconoce como faltantes los objetos de tipo `np.nan`(usado para variables numéricas) y el tipo nativo de Python `None`.  Usemos, como ejemplo, la serie

```python 
import pandas as pd
import numpy as np
ingredientes = pd.Series(['ajo', 'cebolla', 'pimiento', np.nan, 'sal', 'pimienta', None])
```

En pandas contamos con cuatro métodos básicos para lidiar con los datos ausentes

Método|Desripción
---|---
isna(a.k.a isnull)|Vector lógico indicando los elementos ausentes como `True`
notna (a.k.a notnull)| Negación lógica de `isnull`
dropna| Filtra la base por la presencia de valores ausentes. Se puede modular el umbral de tolerancia.
fillna| Imputación de valores en las observaciones ausentes. Puede ser un valor específico o un método de interpolación

```python
print(ingredientes.isna())
print(ingredientes.notna())
print(ingredientes.dropna())
print(ingredientes.fillna('Sabe qué cosa'))
```

En una base, es decir, cuando tenemos más de una columna, hay más posibilidades:

```python
from numpy import nan as NA
datos = pd.DataFrame([[1, 2, 3], [4, NA, NA], [NA, 5, 6], [NA, NA, NA]])
print(datos, '\n\n')
print(datos.dropna(), '\n\n') # elimina todos los renglones que contienen al menos un NaN
print(datos.dropna(axis = 1),'\n\n') # elimina todas las columnas que contienen al menos un NaN
print(datos.dropna(how = 'all'), '\n\n') # elimina un renglón si todas sus entradas son NaN
print(datos.dropna(thresh = 2), '\n\n') # conserva los renglones con al menos 2 valores observado
print(datos.dropna(subset = [0, 1]))
```

### Tu turno

Usa los datos de `sueño` para responder estas preguntas:
1. Cuántas horas sueña (`sleep_rem`) un mamífero en promedio? Cuál es la mediana? Cuál es el máximo?
2. Cuántos datos faltantes hay en la variable `sleep_rem`? Qué porcentaje de las entradas de sueño total (`sleep_total`) son datos faltantes? Qué porcentaje de las observaciones sobre la forma de alimentación (`vore`) son genuinas?
3. Cuál es el total de observaciones faltantes en el juego de datos?

In [30]:
print(datos, '\n\n')
print(datos.dropna(), '\n\n') # elimina todos los renglones que contienen al menos un NaN
print(datos.dropna(axis = 1),'\n\n') # elimina todas las columnas que contienen al menos un NaN
print(datos.dropna(how = 'all'), '\n\n') # elimina un renglón si todas sus entradas son NaN
print(datos.dropna(thresh = 2), '\n\n') # conserva los renglones con al menos 2 valores observado
print(datos.dropna(subset = [0, 1]))


     0    1    2
0  1.0  2.0  3.0
1  4.0  NaN  NaN
2  NaN  5.0  6.0
3  NaN  NaN  NaN 


     0    1    2
0  1.0  2.0  3.0 


Empty DataFrame
Columns: []
Index: [0, 1, 2, 3] 


     0    1    2
0  1.0  2.0  3.0
1  4.0  NaN  NaN
2  NaN  5.0  6.0 


     0    1    2
0  1.0  2.0  3.0
2  NaN  5.0  6.0 


     0    1    2
0  1.0  2.0  3.0


Otra opción (no necesariamente muy recomendable) es imputar los valores ausentes con el método `.fillna()` que aceepta

Argumento | Acción
---|---
valor | Imputar ese valor
dict / Series / DataFrame | Indicando el valor de reemplazo para cada índice (al aplicarse sobre Series) o columna (al aplicarse sobre DataFrame)
method | Puede ser `backfill` alias `bfill` o `pad` alias `ffill`.  Utilizan las observaciones pasadas o futuras para propagarlas en los valores ausentes
axis | Determina sobre qué eje propagar o reemplazar
limit| Limita la propagación de `bfill` o `ffill` a `limit` observaciones (de tipo `int`)

```python
print(datos, '\n\n')
print(datos.fillna('Reemplazo'), '\n\n')
print(datos.fillna({0:'Cero', 1:'Uno', 2:'Dos'}),'\n\n')
print(datos.fillna(method = 'ffill'), '\n\n')
print(datos.fillna(method = 'ffill', axis = 1), '\n\n')
```



### Tipos de variable

Los tipos de variable que usaremos en nuestros análisis de datos y la información que representan puede leers en la siguiente tabla:

Contenido | Tipo de variable
--- | ---
Conteos, número de unidades, etc. | `int`
Mediciones continuas (salario, temperatura, etc.) | `float`
Clasificación binaria (sexo, afiliación partidaria, etc.) | `bool`
Fechas | `datetime`
Texto | `str`
Categorías (clasificación) | `category`

Una práctica usual es codificar factores como números enteros y dar un diccionario.  En estos casos merece la pena cambiar el tipo de registro de `int` a `category` para obtener información más relevante.  Por ejemplo, miremos la columna `color` de esta base de datos

```python
diamantes = pd.read_csv('../Datos/diamonds_factors.csv', parse_dates = [10])
diamantes
```
El color está codificado con los números 1, 2, 3, ..., 7 y debería ser una variable categórica; pero en nuestra importación es una variable entera.  El resumen `diamantes['color'].describe()` es inútil.  Para arreglarlo simplemente cambiaremos el tipo como sigue:

```python
diamantes['color'] = diamantes['color'].astype('category')
assert diamantes['color'].dtype == 'category'
```
Otro error común de clasificación de tipo ocurre cuando una columna numérica es registrada con sus unidades como en tiempo = _12 minutos_ o precio = $ 450.31.

En estos casos el método `.str.strip('qué_quitar')` puede ser de utilidad.  

### Tu turno
Intenta corregir la columna `precio` de la base de datos de diamantes.  Para ello:
0. Intenta una descripción de la columna de precios. Es la información relevante?
1. Elimina el signo '$' usando el método `str.split('$')`
2. Convierte la columna así obtenida en tipo `float`
3. Escribe un assert statement para verificar este cambio de tipo
4. Vuelve a hacer la descripción.  Los cambios merecen la pena?


## Revisión de límites 

Cierto tipo de datos puede tener límites naturales; por ejemplo, una clasificación podría referir a cinco tipos distintos y con ello su rango estaría limitado a cinco valores.  No obstante, por múltiples razones podría haber registros que no cumplen con estos requisitos. 

En nuestro ejemplo, el campo de fecha de venta tiene un límite natural: la fecha actual.  Para corroborar que todas las fechas de venta han sido correctamente registradas, podemos ejecutar:

```python
import datetime as dt
any(diamantes['date_sold'] > dt.datetime.today())
```
La respuesta `True` nos indica que existen registros fuera de fecha...

### Tu turno
1. Modificando el código anterior, determina cuántos registros están fuera de fecha
2. Imprime solo esos renglones de la base de datos
3. Opciones:

    3.1 Cambia la observación de la columna `date_sold` a NA (`np.nan`) para estas observaciones e imprime de nuevo estos registros
    3.2 En vez de cambiar los registros por NA, redefine la base de forma que incluya solo las observaciones correctamente registradas (Usa el método de selección `diamantes[...]`.
    3.3 Repite el inciso anterior; pero ahora usando el método `.drop()


## Registros duplicados 

Otro problema común en una base de datos es la de registros duplicados.  Para revisar si nuestra base de datos tiene registros duplicados podemos ejecutar.

```python
any(diamantes.duplicated())
```

El vector `diamantes.duplicated()` marca, por defecto, todas menos la primera ocurrencia de la repetición.  En nuestro caso las observaciones 15345, 39878, 41398, 50808, marcadas por el método, corresponden a ocurrencias de registros repetido.  Para ver todas las repeticiones, podemos usar el parámetro `keep = False`.  La otra opción posible es `subset` para elegir las columnas en las que se buscarán los repetidos (id, combinaciones únicas, etc.).

### Tu turno

1. Imprime todas los valores repetidos de la tabla con el método `.duplicated()`.  Asegúrate de ver todas las ocurrencias de cada caso (debe haber dos de cada registro).

2. Ahora ordena tus resultados para mejor visualización con el método `sort_values(by = 'date_sold')

3. Finalmente, utiliza el método `drop_duplicates()` de forma que la base creada conserve únicamente la **última** ocurrencia de cada registro.

Más adelante, conforme desarrollemos más técnicas en Pandas, podremos abordar la solución de problemas más complejos con los datos como inconsistencias en las categorías de registro (tipo de sangre C+), registros múltiples para una misma categoría (como 'Mx', 'mx', 'MX', ' Mx', '      MX   '), exceso de categorías en una clasificación y la necesidad de reasignar grupos, formatos inapropiados en un campo de texto (email: nelson?muriel@gmailcom; teléfono: 4321) y más.

### Preparación: diamantes_clean

Utiliza la siguiente celda para recopilar todos los cambios que hicimos a la base `diamantes` y guarda el resultado final en el objeto `diamantes_clean`.   Tu base final debe tener dimensiones `53940 rows x 11 columns`

In [210]:
diamantes = pd.read_csv('../Datos/diamonds_factors.csv', parse_dates = [10])
diamantes['color'] = diamantes['color'].astype('category')
diamantes['price'] = diamantes['price'].str.strip('$').astype('float')
bad_records = diamantes['date_sold'] > dt.datetime.today()
index = diamantes[bad_records].index
diamantes.drop(index = index)
diamantes_clean = diamantes.drop_duplicates(keep = 'last')
diamantes_clean

,carat,cut,color,clarity,depth,table,price,x,y,z,date_sold
0,1.68,Very Good,3,VS2,61.1,60.0,15309.0,7.63,7.70,4.68,2009-03-30
1,0.38,Ideal,6,SI1,61.8,54.0,593.0,4.66,4.70,2.89,2007-05-10
2,1.20,Fair,4,I1,64.4,55.0,2655.0,6.77,6.61,4.31,2003-01-12
3,0.32,Premium,2,SI1,62.3,58.0,720.0,4.40,4.37,2.73,2007-05-31
4,0.50,Very Good,1,VS2,61.1,58.0,1646.0,5.07,5.11,3.11,2003-08-18
...,...,...,...,...,...,...,...,...,...,...,...
53939,0.54,Ideal,6,SI1,61.4,56.0,1114.0,5.25,5.27,3.23,2011-07-10
53940,0.38,Premium,3,VS2,60.9,56.0,1026.0,4.72,4.67,2.86,2016-05-23
53941,1.07,Ideal,6,VS1,62.0,53.0,5373.0,6.61,6.62,4.10,2011-12-04
53942,1.21,Good,4,SI1,63.8,58.0,5597.0,6.75,6.64,4.27,2006-10-03


## 2. Los métodos de interacción con una tabla.

Hay seis acciones básicas que nos serán de utilidad en el análisis de una tabla:
1. Hacer consultas (`.query()`) sobre las columnas de una base de datos, p.ej. "Restrignirse a los diamantes con una claridad "VS2" y color 3)"
2. Hacer un filtrado de columnas (`.filter()`) para utilizar únicamente las columnas que sean de nuestro interés, p. ej. mirar únicamente el quilataje, color, claridad, y precio de los diamantes.
3. Generar nuevas columnas (`.assign()`) usando la información presente en la base de datos, p.ej. calcular el precio por quilate dividiendo `price` entre `carat`.
4. Agrupar nuestra información en grupos para el análisis (`.groupby()`), p.ej. agrupar por `cut` para examinar el valor en cada categoría.
5. Realizar cálculos agregados (`.agg()`) como medias, desviaciones estándar, etc., p.ej. calcular el precio medio de los diamantes de cada tipo de corte.
6. Ordenar los resultados según una columna (`.sort_values()`).

El código para las consultas especificadas en los ejemplos es el siguiente:

```python
diamantes_clean.query('clarity == "VS2" and color == 3')
diamantes_clean.filter(['carat', 'color', 'clarity', 'price'])
diamantes_clean.assign(ppcarat = lambda x: x.price/x.carat)
diamantes_clean.groupby('cut') #en realidad no cambia nada, solo agrupa
diamantes_clean.groupby('cut').agg(mean)
```
Obsérvese que en las últimas línea se concatenaron dos métodos. Como consejo de estilo, puede ser más legible el escribir esta consulta en la forma: 

```python
(diamantes_clean
  .groupby('cut')
  .agg('mean'))
```

Ahora bien, podríamos concatenar un tercero si quisieramos exacta y únicamente los precios medios...

Algunas funciones que podemos usar dentro de `.agg()` son:

Función | Descripción
---|---
mean | Calcula la media (promedio aritmético)
median | Calcula la mediana (percentil 50%)
var | Calcula la varianza muestral
std | Calcula la desviación estándar
sum | Suma los elementos
quantile | Calcula los cuantiles (percentiles) de la variable
min | Calcula el valor mínimo
max | Calcula el valor máximo
count | Cuenta (generalmente usado sobre grupos o categorías)
describe | Funciona también concatenado a `groupby`

### Tu Turno
1. Para ver más de una agregación, podemos usar una lista como argumento para `.agg()`.  Completa la última consulta añadiendo `median` y `std` a las funciones de agregación y el método `.filter()` a la consulta de modo que sólo se reporte el precio.  Recuerda que se filtra con una **lista**.  

2. Otra opción en `.agg()` es definir tu propia función de agregación.  En este caso debemos pasar **el nombre de la función** directamente, **no en tipo** `str`.  Define tu propia función `iqr()` basada en `np.quantile` y añade esta función al resumen.  Asigna esta base al objeto `price_summary`.

#### Nota

Esta base de datos tiene un índice jerárquico para las columnas.  Las columnas ya no son `price_summary['mean']` ni `price_summary.loc[:, 'mean']`.  Ahora deberemos acceder a ellas con una `tuple` para el nombre de columna.  La primera entrada es el identificador exterior (`price`) y la segunda entrada es el identificador interior (`mean`, `median`, `std`, `iqr`).  Por ejemplo, podemos seleccionar
```python
price_summary.loc[: , ('price', ['mean'])]
```
Observe que `'mean'` aparece dentro de corchetes.  El seleccionar con una lista provoca que la respuesta sea del tipo `pandas.core.frame.DataFrame` y no del tipo `pandas.core.series.Series`.  En caso de brindar una lista más larga, la selección será de esas columnas.

##### Práctica

A modo de práctica:
1. Genera una base de datos en la que se lea la media, desviación estándar y mediana para las variables `price` y `carat` (en ese orden) agrupados por la variable `cut`.
2. Selecciona, de la base anterior, las columnas media y mediana para `price`.

## 2.1 Usando los métodos de interacción de una tabla 

Estos cinco métodos son la base para responder preguntas concretas sobre la información presente en la base de datos. Para ejemplificar esto, usaremos la base de datos `msleep`, proveniende del paquete `ggplot2` de R y disponible en `https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/msleep.csv`.   Consulte la [descripción de esta base](https://ggplot2.tidyverse.org/reference/msleep.html).  

Preguntas: 
1. Hay alguna diferencia en el tiempo total del sueño según el tipo de alimentación?
2. Algún régimen alimenticio está asociado, en esta base, con cerebros más grandes (relativos al cuerpo)?
3. Cuáles son los cinco animales que más tiempo duermen, qué porcentaje del día duermen, y qué dieta siguen?

Respuestas posibles:
```python
# 1. -------
(
 msleep
    .filter(['sleep_total'])
    .groupby('vore')
    .agg(['mean', 'median', 'std', iqr])
)


# 1 extendida. -------

def perc_over_10(x):
    return np.mean(x >= 10)

def perc_over_15(x):
    return np.mean(x >= 15)

(
msleep
    .filter(['vore', 'sleep_total'])
    .groupby('vore')
    .agg(['mean', 'median', iqr, perc_over_10, perc_over_15])
)


# 2. -------
brainwt_data = (msleep
    .filter(['vore', 'brainwt', 'bodywt'])
    .assign(percentage_brainwt = lambda x: 100 * x.brainwt / x.bodywt)
    .groupby('vore')
    .agg(['mean', 'median', 'std', iqr, 'count']))

brainwt_data.loc[:, ['percentage_brainwt']]


# 3. ------

(msleep
 .filter(['name', 'sleep_total','vore'])
 .assign(sleep_relative_day = lambda x: 100 * x.sleep_total / 24)
 .sort_values(by = 'sleep_total', ascending = False)
 .iloc[0:5, ]
)

```
En la siguiente celda tienes el código para cargar la base de datos.  Utiliza las siguientes celdas para probar estas consultas. 

### Tu turno
Trabajemos en grupo... Cada grupo deberá plantear 5 preguntas a su base de datos y responderlas con las 6 operaciones básicas.




In [431]:
url = 'https://vincentarelbundock.github.io/Rdatasets/csv/ggplot2/msleep.csv'
msleep = pd.read_csv(url, index_col = 0)

In [440]:
# Pega aquí las consultas que quieras ejeutar

## Reshaping your data...otras formas de visualización y comunicación

### 1. Pivotar
Pandas ofrece distintos métodos para cambiar la forma de nuestros datos.  Exploremos primero con _pivotar_ tablas **sin agregar valores** sino únicamente cambiando de forma.
```python
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two','two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})

df
df.pivot(index = 'foo', columns = 'bar', values = 'baz')
df.pivot(index = 'foo', columns = 'bar', values = ['baz', 'zoo'])
df.pivot(index = 'foo', columns = 'bar')
```
El método no permite repeticiones y devolverá un `ValueError` si los encuentra como en

```python
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo", "bar", "bar", "bar", "bar"],
                        "B": ["one", "one", "one", "two", "two", "one", "one", "two", "two"],
                        "C": ["small", "large", "large", "small", "small", "large", "small", "small","large"],
                        "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                        "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df.pivot(index = 'A', columns = 'B', values = 'C')
```

Para casos como este, tenemos el método `.pivot_table()` que además de los argumentos `index`, `columns`, y `values`, utiliza el argumento `aggfunc`, la función para agregar los casos repetidos. Su valor por defecto es `numpy.mean`. Puede utilizarse una lista de funciones y el resultado será una tabla indexada jerarquicamente. 

```python
df.pivot_table(index = 'A', columns = 'B', values = 'C', aggfunc = 'count')
df.pivot_table(index = 'A', columns = 'B', values = 'D', aggfunc = 'sum')
df.pivot_table(index = 'A', columns = 'B', values = 'D', aggfunc = ['mean', 'std'])
df.pivot_table(index = 'A', columns = 'B', values = ['D', 'E'], aggfunc = ['mean', 'std'])
df.pivot_table(index = 'A', columns = 'B', values = ['D', 'E'], aggfunc = ['mean', 'std'], margins = True)
df.pivot_table(index = 'A', columns = 'B', aggfunc = ['mean', 'std'])  # Falta algo?
df.pivot_table(index = 'A', columns = 'B', values = 'C', aggfunc = ['mean', 'std'])  # DataError

df.pivot_table(index = ['A', 'B'], columns = 'C', values = 'D')
```

In [4]:
import pandas as pd
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two','two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df.pivot(index = 'foo', columns = 'bar', values = 'baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


### 2. Lo contrario...expandiendo nuestra base

Pivotar la tabla produce una compresión de los datos. La operación contraria, expandir la base para ver todos sus registros, se puede llevar a cabo con el método `.melt()`.  Es un método útil cuando los nombres de columna son valores de una variable; esto es, cuando la tabla está pivotada.  


In [26]:
import numpy as np
from datetime import datetime

stocks = pd.DataFrame({'X':np.random.normal(size = 10), 'Y': np.random.normal(scale = 2, size = 10),
                       'Z':np.random.normal(size = 10, scale = 4)})
stocks.index = pd.date_range(datetime.today(), periods = 10, freq = '5H')
stocks.head()

,X,Y,Z
2020-09-02 10:41:48.285833,1.012401,-0.525320,-2.206493
2020-09-02 15:41:48.285833,-0.196708,1.994933,-0.020539
2020-09-02 20:41:48.285833,-0.815031,-0.979125,3.615174
2020-09-03 01:41:48.285833,-1.620917,0.333095,9.185839
2020-09-03 06:41:48.285833,-0.003679,1.901970,1.223905


El método `.melt()` utiliza los siguientes argumentos:

Argumento |  Uso
---| --- 
id_vars |  Son las columnas de la base que no están pivotadas, aquéllas que son valores observados en sí
value_vars | Refiere a las columnas de la base que contienen variables 
value_name | Tras el reacomodo, cómo se llama la variable que está registrada en los valores de las columnas pivotadas (qué representa?)
var_name | Nombre de la variable que está registrada en las columnas

In [36]:
stocks.melt(value_vars = ['X', 'Y', 'Z'], var_name = 'Stock', value_name = 'Retorno')

,Stock,Retorno
0,X,1.012401
1,X,-0.196708
2,X,-0.815031
3,X,-1.620917
4,X,-0.003679
5,X,1.049084
6,X,0.851669
7,X,0.895696
8,X,0.801915
9,X,0.837260


#### Tu turno: Reorganizando un juego de datos más grande...
El Secretariado Ejecutivo del Sistema Nacional de Seguridad publica los datos de la incidencia delictiva del fuero común por estado. Para leerlos, hacemos:

In [ ]:
import pandas as pd
incidencia_fc = pd.read_csv('../Datos/IDEFC_NM_may2020.csv', encoding = 'iso-8859-1', thousands = ',')
incidencia_fc.head()

Identifica cuáles variables necesitamos reestructurar en el método melt y arregla la base de datos para que esté en un estado *tidy*.